# Tutorial: Integrating GraphSpace into network analysis projects
See session2.ipynb to get set up.

About Me:
  - Jeff Law
  - PhD Student, Genetics, Bioinformatics and Computational Biology at Virginia Tech
  - Email: jeffl@vt.edu

## Agenda
1. Pipeline Overview
2. Introduction to PathLinker
  - Protein-protein interaction network
  - k shortest paths
3. Post results to GraphSpace
  - Read in PathLinker results
  - Add style, popups, path rank filter and more attributes
4. Reconstruction of the WNT Signaling Pathway
5. Post your own networks!

### Introduction to PathLinker
PathLinker is an implementation of the k shortest paths algorith known as [Yen's Algorithm](https://en.wikipedia.org/wiki/Yen%27s_algorithm).
PathLinker has some useful additional features
  - shortest paths from multiple sources to multiple targets

<p><a href="https://commons.wikimedia.org/wiki/File:Yen%27s_K-Shortest_Path_Algorithm,_K%3D3,_A_to_F.gif#/media/File:Yen%27s_K-Shortest_Path_Algorithm,_K%3D3,_A_to_F.gif"><img src="https://upload.wikimedia.org/wikipedia/commons/d/d1/Yen%27s_K-Shortest_Path_Algorithm%2C_K%3D3%2C_A_to_F.gif" alt="Yen's k-shortest path algorithm, K&nbsp;=&nbsp;3, A to F"></a><br>By <a href="//commons.wikimedia.org/w/index.php?title=User:KRPent&amp;action=edit&amp;redlink=1" class="new" title="User:KRPent (page does not exist)">KRPent</a> - Using an open source graphing library, Graphviz, to generate the image., <a href="http://creativecommons.org/licenses/by-sa/3.0" title="Creative Commons Attribution-Share Alike 3.0">CC BY-SA 3.0</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=22926392">Link</a></p>

## Run PathLinker
First we need to download PathLinker

In [1]:
!git clone "https://github.com/Murali-group/PathLinker"

Cloning into 'PathLinker'...
remote: Counting objects: 168, done.
remote: Total 168 (delta 0), reused 0 (delta 0), pack-reused 168
Receiving objects: 100% (168/168), 2.68 MiB | 0 bytes/s, done.
Resolving deltas: 100% (72/72), done.
Checking connectivity... done.


### Protein-protein interaction network
Protein-protein interactions (PPIs) are available from multiple databases for many species. 
We work with Human and Yeast 

Sources: BioGrid, DIP, InnateDB, IntAct, KEGG, MatrixDB, MINT, NetPath, PhosphositePlus, Reactome, SPIKE'
  - 35K regulatory (directed) PPIs
  - 200K physical (undirected) PPIs
  
Edges are weighted using an evidence-based probabilistic approach.

### Sources and Targets
Using the WNT pathway receptors as sources and the transcription facotrs (TFs) as targets, we can use PathLinker to build a reconstruction of the WNT pathway.

### Running PathLinker
Now that we have all of the inputs set, we can run PathLinker



In [2]:
!python PathLinker/PathLinker.py --help

Usage: 
PathLinker.py [options] NETWORK NODE_TYPES
REQUIRED arguments:
    NETWORK - A tab-delimited file with one directed interaction per
        line. Each line should have at least 2 columns: tail, head. Edges
        are directed from tail to head. This file can have a third column
        specifying the edge weight, which is required unless the --PageRank
        option is used (see --PageRank help for a note on these weights).
        To run PathLinker on an unweighted graph, set all edge weights
        to 1 in the input network.

    NODE_TYPES - A tab-delimited file denoting nodes as receptors or TRs. The first
        column is the node name, the second is the node type, either 'source'
        (or 'receptor') or 'target' (or 'tr' or 'tf'). Nodes which are neither receptors nor TRs may
        be omitted from this file or may be given a type which is neither 'source'
        nor 'target'.
    


Options:
  -h, --help            show this help message and exit
  -o STR, --out

In [5]:
%%bash
mkdir -p output
python PathLinker/PathLinker.py \
    data/2017-06-16-human-ppi-weighted-cap0_75.tsv \
    data/Wnt-sources-targets.tsv \
    -k 100 \
    -o output/Wnt- \
    --write-paths 


Reading the network from data/2017-06-16-human-ppi-weighted-cap0_75.tsv
Name: 
Type: DiGraph
Number of nodes: 16421
Number of edges: 438775
Average in degree:  26.7204
Average out degree:  26.7204
Reading sources and targets from data/Wnt-sources-targets.tsv

Read 14 sources and 14 targets
	After removing sources and targets that are not in the network: 14 sources and 14 targets.

Computing the k=100 shortest simple paths.

KSP results are in "output/Wnt-k_100-ranked-edges.txt"
KSP paths are in "output/Wnt-k_100-paths.txt"

Finished!


In [8]:
!head output/Wnt-k_100-paths.txt output output/Wnt-k_100-ranked-edges.txt

==> output/Wnt-k_100-paths.txt <==
#KSP	path_length	path
1	7.50000e-01	O75581|P98082
2	5.62500e-01	O00144|O14640|P25054
3	5.62500e-01	O00144|O14640|P35222
4	5.62500e-01	O00144|O14640|Q15797
5	5.62500e-01	O00144|O14641|P25054
6	5.62500e-01	O00144|Q92997|P98082
7	5.62500e-01	O75084|O14640|P25054
8	5.62500e-01	O75084|O14640|P35222
9	5.62500e-01	O75084|O14640|Q15797

==> output <==
head: error reading 'output': Is a directory

==> output/Wnt-k_100-ranked-edges.txt <==
#tail	head	KSP index
O75581	P98082	1
O00144	O14640	2
O14640	P25054	2
O14640	P35222	3
O14640	Q15797	4
O00144	O14641	5
O14641	P25054	5
O00144	Q92997	6
Q92997	P98082	6


## Post results to GraphSpace
This step is not too difficult if we just want to view the nodes, edges and paths on GraphSpace. However, there is much more information we have that we would like to be able to quickly access and see on GraphSpace. For example:
  - What sources of evidence do we have for each interaction in the top 100 paths?
  - Which proteins and PPIs in the top 100 paths are known to be part of the NetPath or KEGG WNT pathway?

In [10]:
# Make sure graphspace-python library is installed
!pip install graphspace_python

In [11]:
# Read in the PathLinker results and keep track of the k for each of the edges
ranked_edges = {}
with open('output/Wnt-k_100-ranked-edges.txt', 'r') as file_handle:
    for line in file_handle:
        if line[0] == '#':
            continue
        line = line.rstrip().split('\t')
        edge = (line[0], line[1])
        k = line[2]
        ranked_edges[edge] = k
        
# Read in the sources and targets file so we can change the shape and color of the sources and targets
sources = set()
targets = set()
with open('data/Wnt-sources-targets.tsv', 'r') as file_handle:
    for line in file_handle:
        if line[0] == '#':
            continue
        line = line.rstrip().split('\t')
        prot = line[0]
        prot_type = line[1]
        if prot_type.lower() in ["receptor", "source"]:
            sources.add(prot)
        elif prot_type.lower() in ["tf", "target"]:
            targets.add(prot)

print("%d edges, %d sources, %d targets" % (len(ranked_edges), len(sources), len(targets)))

94 edges, 14 sources, 14 targets


In [38]:
# before adding these edges to a GraphSpace object, 
# we need to get the node, edge, and pathway information we want to post
# we also need to be able to map the uniprot IDs to their more friendly gene names
uniprot_to_gene = {}
with open('data/2017_06-human-interactome-mapping.tsv', 'r') as file_handle:
    for line in file_handle:
        if line[0] == '#':
            continue
        uniprotID, gene_name = line.rstrip().split('\t')
        uniprot_to_gene[uniprotID] = gene_name
        
# get the edge weights from the interactome
edge_weights = {}
with open('data/2017-06-16-human-ppi-weighted-cap0_75.tsv', 'r') as file_handle:
    for line in file_handle:
        if line[0] == '#':
            continue
        u, v, w, evidence = line.rstrip().split('\t')
        edge_weights[(u,v)] = float(w)
        
print("%s interactome edges" % (len(edge_weights)))
        
# get the netpath and kegg edges and nodes
# KEGG WNT pathway: http://www.genome.jp/kegg-bin/show_pathway?hsa04310
# NetPath WNT pathway: http://www.netpath.org/pathways?path_id=NetPath_8
kegg_wnt_edges = set()
netpath_wnt_edges = set()
evidence_lines = 'data/2017_06-human-interactome-evidence.tsv'
with open(evidence_lines, 'r') as file_handle:
    for line in file_handle:
        if line[0] == "#":
            continue
        u,v, directed, interactiontype, detectionmethod, pubid, source = line.rstrip().split('\t')
        if source.lower() == 'kegg' and pubid.lower() == 'kegg:hsa04310':
            kegg_wnt_edges.add((u,v))
            if directed == "False":
                kegg_wnt_edges.add((v,u))
        elif source.lower() == 'netpath' and pubid.lower() == "netpath:netpath_8":
            netpath_wnt_edges.add((u,v))
            if directed == "False":
                netpath_wnt_edges.add((v,u))
kegg_wnt_nodes = set([n for u,v in kegg_wnt_edges for n in (u,v)])
netpath_wnt_nodes = set([n for u,v in netpath_wnt_edges for n in (u,v)])

print("%s kegg wnt nodes, %s kegg wnt edges" % (len(kegg_wnt_nodes), len(kegg_wnt_edges)))
print("%s netpath wnt nodes, %s netpath wnt edges" % (len(netpath_wnt_nodes), len(netpath_wnt_edges)))

438775 interactome edges
132 kegg wnt nodes, 940 kegg wnt edges
81 netpath wnt nodes, 245 netpath wnt edges


In [20]:
# this next section is to get the sources of evidence, interaction type, and pubmed IDs for each interaction
# this is a bit more difficult, but we can load it into a multi-level dictionary
def getEvidence(edges, evidence_file):
    """
    *edges*: a set of edges for which to get the evidence for
    *split_family_nodes*: add evidence of family edges to pairwise interactions 
    *add_ev_to_family_edges*: add evidence of PPI edge covered by a family edge to the family edge. 
                              If this option is specified, then a dictionary of direct evidence along with
                              a dictionary of the ppi evidence applied to the family edge is returned
    returns a multi-level dictionary with the following structure
    edge: 
      db/source: 
        interaction_type: 
          detection_method: 
            publication / database ids
    for NetPath, KEGG and SPIKE, the detection method is the pathway name 
    NetPath, KEGG, and Phosphosite also have a pathway ID in the database/id set
    which follows convention of id_type:id (for example: kegg:hsa04611)
    """
    # initialize the evidence, edge_types, and edge dir/undir dictionaries
    evidence = {}
    edge_types = {}
    edge_dir = {}

    print "Reading evidence file %s" % (evidence_file)

    # create a graph of the passed in edges 
    G = nx.Graph()
    G.add_edges_from(edges)

    # initialize the dictionaries
    for t,h in G.edges():
        evidence[(t,h)] = {}
        evidence[(h,t)] = {}
        edge_types[(t,h)] = set()
        edge_types[(h,t)] = set()
        edge_dir[(t,h)] = False
        edge_dir[(h,t)] = False

    file_handle = open(evidence_lines, 'r')
    for line in file_handle:
        if line[0] == "#":
            continue
        u,v, directed, interactiontype, detectionmethod, pubid, source = line.rstrip().split('\t')
        # header line of file
        #uniprot_a  uniprot_b   directed    interaction_type    detection_method    publication_id  source
        directed = True if directed == "True" else False

        # We only need to get the evidence for the edges passed in, so if this edge is not in the list of edges, skip it
        # G is a Graph so it handles undirected edges correctly
        if not G.has_edge(u,v):
            continue

        evidence = addToEvidenceDict(evidence, (u,v), directed, source, interactiontype, detectionmethod, pubid)
        edge_types = addEdgeType(edge_types, (u,v), directed, source, interactiontype)

        if directed is True:
            edge_dir[(u,v)] = True
        else:
            # if they are not already directed, then set them as undirected
            if (u,v) not in edge_dir:
                edge_dir[(u,v)] = False
                edge_dir[(v,u)] = False

    return evidence, edge_types, edge_dir


    
def addToEvidenceDict(evidence, e, directed, source, interactiontype, detectionmethod, pubid):
    """ add the evidence of the edge to the evidence dictionary
    *pubids*: publication id to add to this edge. 
    """
    if source not in evidence[e]:
        evidence[e][source] = {}
    if interactiontype not in evidence[e][source]:
        evidence[e][source][interactiontype] = {}
    if detectionmethod not in evidence[e][source][interactiontype]:
        evidence[e][source][interactiontype][detectionmethod] = set()
    evidence[e][source][interactiontype][detectionmethod].add(pubid)

    if not directed:
        # add the evidence for both directions
        evidence = addToEvidenceDict(evidence, (e[1],e[0]), True, source, interactiontype, detectionmethod, pubid)

    return evidence


def addEdgeType(edge_types, e, directed, source, interactiontype):
    if e not in edge_types:
        edge_types[e] = set()
    # add the edge type as well
    # direction was determined using the csbdb_interface.psimi_interaction_direction dictionary in CSBDB. 
    if not directed:
        # it would be awesome if we knew which edges are part of complex formation vs other physical interactions
        # is there an mi term for that?
        t,h = e
        edge_types[(t,h)].add('physical')
        edge_types[(h,t)].add('physical')

    elif source == "SPIKE":
        edge_types[e].add('spike_regulation')

    elif source == "KEGG":
        if "phosphorylation" in interactiontype and "dephosphorylation" not in interactiontype:
            # Most of them are phosphorylation
            edge_types[e].add('phosphorylation')
        if "activation" in interactiontype:
            edge_types[e].add('activation')
        if "inhibition" in interactiontype:
            edge_types[e].add('inhibition')
        else:
            edge_types[e].add('enzymatic')
    else:
        # the rest is psi mi tags
        # MI:0217 is for phosphorylation
        if "MI:0217" in interactiontype:
            # Most of the directed edges are phosphorylation
            edge_types[e].add('phosphorylation')
        else:
            # TODO for now, just call the rest of the directed edges enzymatic. 
            edge_types[e].add('enzymatic')

    return edge_types

In [44]:
# converts the evidence dictionary to html for the popup
def evidenceToHTML(u,v,evidence):
    annotation = '<dl>'
    sources = sorted(evidence.keys())
    for source in sources:
        annotation += '<dt>%s</dt>' % (source)
        # TODO add interaction type color
        for interactiontype in evidence[source]:
            if interactiontype != '' and interactiontype != "None":
                # use bull instead of li to save on white space
                # nbsp stands for non-breaking space
                annotation += '&bull;&nbsp&nbsp%s <br>' % interactiontype
            for detectionmethod in evidence[source][interactiontype]:
                # add a bullet point here after 4 spaces
                annotation += '&nbsp&nbsp&nbsp&nbsp&bull;&nbsp&nbsp'
                annotation += '%s  ' % detectionmethod

                # now add the pubmed IDs. &nbsp is the html for a non-breaking space
                pub_ids = evidence[source][interactiontype][detectionmethod]
                #KEGG doesn't have pub ids. It has a pathway map and entry (evidence)
                # now get the html for each of the links
                pub_ids = [parsePubID(pub_id) for pub_id in pub_ids if parsePubID(pub_id) != '']

                # use a non-breaking space with a comma so they all stay on the same line
                annotation += ',&nbsp'.join(pub_ids)
                annotation += "<br>"
        annotation += '<br>'
    return annotation

# adds the links to the pubmed and other IDs
def parsePubID(publication_id):
    id_type, pubid = publication_id.split(':')
    if id_type == 'pubmed':
        pubmedurl = 'http://www.ncbi.nlm.nih.gov/pubmed/%s' % (pubid)
        desc = '<a style="color:blue" href="%s" target="PubMed">pmid:%s</a>' % (pubmedurl,pubid)
    elif id_type == 'phosphosite':
        phosphourl = 'http://www.phosphosite.org/siteAction.action?id=%s' % (pubid)
        desc = '<a style="color:blue" href="%s" target="PSP">phosphosite:%s</a>' % (phosphourl,pubid)
    elif id_type == 'kegg':
        # links to KEGG pathway map
        kegg_map_link = 'http://www.kegg.jp/kegg-bin/show_pathway?'
        # links to KEGG pathway entry (evidence)
        kegg_entry_link = 'http://www.kegg.jp/dbget-bin/www_bget?pathway+'
        pathway_map_link = '<a style="color:blue" href="%s%s" target="KEGG">map</a>' % (kegg_map_link,pubid)
        pathway_entry_link = '<a style="color:blue" href="%s%s" target="KEGG">evidence</a>' % (kegg_entry_link,pubid)
        desc = "%s,&nbsp%s"%(pathway_map_link,pathway_entry_link)
    elif id_type == 'netpath':
        netpath_url = "http://www.netpath.org/reactions?path_id=%s" % (pubid)
        # links to KEGG pathway entry (evidence)
        desc = '<a style="color:blue" href="%s" target="NetPath">netpath:%s</a>' % (netpath_url, pubid)
    else:
        # skip the rest for now
        desc = ''

    return desc



In [22]:
# takes in a node ID and the list of paths its in, and returns the html popup
def getNodePopup(n, pathswithnode):
    htmlstring = ''
    uniproturl = 'http://www.uniprot.org/uniprot/%s' 
    htmlstring += '<b>Uniprot ID</b>: <a style="color:blue" href="%s" target="UniProtKB">%s</a><br>' % (uniproturl%n, n)
    htmlstring += '<hr />'

    htmlstring += '<b>Paths</b>: %s<br>' %(','.join(str(k) for k in sorted(pathswithnode)))
    
    return htmlstring


# takes in the u->v edge, first path index k its in, and sources of evidence 
# returns the html edge popup
def getEdgePopup(u, v, k, evidence):
    annotation = '' 
    annotation += '<b>%s - %s</b></br>'%(uniprot_to_gene[u], uniprot_to_gene[v])
    annotation += '<b>%s - %s</b></br>'%(u,v)
    annotation += '<b>Weight</b>: %.3f</br>' % (edge_weights[(u,v)])
    annotation += '<b>Edge Ranking</b>: %s' % (k)

    annotation += '<hr /><h><b>Sources of Evidence</b></h>'
    annotation += evidenceToHTML(u,v,evidence[(u,v)])

    return annotation



In [1]:
# Dictionaries of node and edge properties
NODE_COLORS = {
        #'target'  : '#FFFF60',  # yellow
        #'source'  : '#8CE1DC',  # light blue
        'target'  : '#4286f4',  # blue
        'source'  : '#4286f4',  # blue
        'default' : '#D8D8D8',  # gray
        'kegg'    : '#ad6cfc',  # purple
        'netpath' : '#4286f4',  # blue
}
NODE_SHAPES = {
        'source'  : 'triangle',
        'target'  : 'rectangle',
        'default' : 'ellipse',
}
EDGE_COLORS = {
        'physical'        : '#27AF47',  # green
        'phosphorylation' : '#F07406',  # orange
        #'enzymatic'       : '#2A69DC',  # blue
        'enzymatic'       : '#DD4B4B',  # red
        'activation'      : 'grey',
        'inhibition'      : 'grey',
        'spike_regulation': 'brown',
        'kegg'            : '#ad6cfc', # purple
        'netpath'         : '#4286f4',  # blue
}

# Adds all of the nodes and edges to the GraphSpace object with the specified shapes and colors
def constructGraph(ranked_edges, sources, targets):
    '''
    Posts the pathlinker result to graphspace

    :param sources: list of source nodes
    :param targets: list of target nodes
    '''
    # NetworkX object
    #G = nx.DiGraph(directed=True)
    G = GSGraph()


    evidence, edge_types, edge_dir = getEvidence(ranked_edges.keys(), evidence_file)

    nodes = set([n for u,v in ranked_edges for n in (u,v)])

    ## add GraphSpace/Cytoscape.js attributes to all nodes.
    for n in nodes:
        #default is gray circle
        node_type = 'default'
        if n in sources:
            # if n is the source, make it a triangle
            node_type = 'source'
        elif n in targets:
            # if n is a taret, make it a square
            node_type = 'target'

        edgeswithnode = set([(t,h) for t,h in ranked_edges if t==n or h==n])
        pathswithnode = set([int(ranked_edges[e]) for e in edgeswithnode])
        k_value = min(pathswithnode)
        # set the name of the node to be the gene name and add the k to the label
        gene_name = uniprot_to_gene[n]
        node_popup = getNodePopup(n, pathswithnode)
        label="%s\n%d"%(gene_name,k_value)

        G.add_node(gene_name, popup=node_popup, label=label, k=k_value)

        # now add the style for the node
        shape = NODE_SHAPES[node_type]
        color = NODE_COLORS[node_type]
        if n in netpath_wnt_nodes:
            color = NODE_COLORS['netpath']
        elif n in kegg_wnt_nodes:
            color = NODE_COLORS['kegg']

        G.add_node_style(gene_name, shape=shape, color=color, width=45, height=45,
                         style='solid', border_color=color, border_width=2, bubble=color)

    # Add all of the edges and their Graphspace/Cytoscape.js attributes
    for (u,v) in ranked_edges:
        # get the main edge type
        main_edge_type = getMainEdgeType(u,v,edge_types[(u,v)])

        gene_name_u = uniprot_to_gene[u]
        gene_name_v = uniprot_to_gene[v]
        edge_popup = getEdgePopup(u,v,k_value, evidence)
        k_value = ranked_edges[(u,v)]

        G.add_edge(gene_name_u,gene_name_v,directed=edge_dir[(u,v)],popup=edge_popup,k=k_value)

        # some attributes (such as opacity) are not implemented in graphspace-python. 
        # For those, we need to make our own dictionary of the attribute and value
        attr_dict = {}
        #attr_dict['opacity'] = 0.8
        color = EDGE_COLORS[main_edge_type]
        if (u,v) in netpath_wnt_edges:
            color = EDGE_COLORS['netpath']
        elif (u,v) in kegg_wnt_edges:
            color = EDGE_COLORS['kegg']
        # use the edge weight to set the width of the edge
        width = edge_weights[(u,v)] * 2
        if 'activation' not in edge_types[(u,v)] and 'inhibition' in edge_types[(u,v)]:
            arrow_shape = 'tee'
        else:
            arrow_shape = 'triangle'

        G.add_edge_style(gene_name_u, gene_name_v, attr_dict=attr_dict,
                         directed=edge_dir[(u,v)], color=color, width=1.5, arrow_shape=arrow_shape,edge_style='solid')
    return G


def getMainEdgeType(u,v, edge_types):
    """ a single edge can have multiple edge types according to the different sources or databases
    Choose a main edge type here
    *edge_types* the set of edge types for a given edge 
    """
    main_edge_type = None
    edge_type_order = ['phosphorylation', 'enzymatic', 'spike_regulation', 'activation', 'inhibition', 'physical']
    for edge_type in edge_type_order:
        if edge_type in edge_types:
            main_edge_type = edge_type 
            break
            
    if main_edge_type is None:
        print("Warning: edge type of %s->%s not found. Setting to activation" % (u,v))
        main_edge_type = 'activation'

    return main_edge_type


In [70]:
from graphspace_python.api.client import GraphSpace
from graphspace_python.graphs.classes.gsgraph import GSGraph
import networkx as nx

# now finally construct the GraphSpace graph and post it to GraphSpace!
G = constructGraph(ranked_edges, sources, targets)

# TODO add a description of the graph with a legend describing shapes and colors
desc = ''
tags = ['pathlinker', 'netpath', '2017-icsb-tutorial']
title = 'NetPath WNT Pathway PathLinker Reconstruction'
metadata = {'description':desc,'tags':tags, 'title':title}
G.set_data(metadata)

graph_name = "netpath-wnt-pathlinker-k100"
G.set_name(graph_name)
    
# post to graphspace
gs = GraphSpace('user6@example.com', 'user6')
gs_graph = gs.get_graph(graph_name, owner_email='user6@example.com')
if gs_graph is None:
    print("\nPosting graph '%s' to graphspace\n" % (graph_name))
    gs_graph = gs.post_graph(G)
else:
    print("\nGraph '%s' already exists. Updating it\n" % (graph_name))
    gs_graph = gs.update_graph(graph_name, owner_email='user6@example.com', graph=G)


Reading evidence file data/2017_06-human-interactome-evidence.tsv

Graph 'netpath-wnt-pathlinker-k100' already exists. Updating it



## Reconstruction of the WNT Signaling Pathway
![NetPath-WNT-PathLinker-k100](images/NetPath-WNT-PathLinker-k100.png)

- First non-WNT pathay node is CFTR
  - Experimentally validated RYK-CFTR-DAB2 WNT signal
  - Ritz et al., Pathways on demand: automated reconstruction of human signaling networks. npj: Systems Biology and Applications, 2016.

## Post your own networks!